In [1]:
!pip install bert-score
!gdown --id '1lovF1aJoy6IV2LpfeKO-NrObVGjED2Mv'
!pip install googletrans==3.1.0a0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1lovF1aJoy6IV2LpfeKO-NrObVGjED2Mv
To: /content/hcm.xlsx
100% 46.7M/46.7M [00:02<00:00, 23.0MB/s]
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from googletrans import Translator
import bert_score
import torch
translator = Translator()

In [3]:
def append_record_to_excel(file_path, question,answer,fa_question,fa_answer,
                           bt_question,bt_answer,question_P, question_R,
                           question_F1,answer_P, answer_R, answer_F1):
    new_record = {
        'question': question,
        'answer': answer,
        'fa_question':fa_question,
        'fa_answer':fa_answer,
        'bt_question':bt_question,
        'bt_answer': bt_answer,
        'question_P': question_P,
        'question_R': question_R,
        'question_F1': question_F1,
        'answer_P': answer_P,
        'answer_R': answer_R,
        'answer_F1':answer_F1
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)

In [ ]:
bgn = 2013
bsize = 100
df = pd.read_excel('hcm.xlsx')
df = df[bgn:]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
questions = []
answers = []
fa_questions = []
fa_answers = []
bt_questions = []
bt_answers = []
for index,row in df.iterrows():
    question = row['question']
    questions.append(question)
    answer = row['answer']
    answers.append(answer)
    fa_question = translator.translate(question, dest='fa').text
    fa_questions.append(fa_question)
    fa_answer = translator.translate(answer, dest='fa').text
    fa_answers.append(fa_answer)
    bt_question = translator.translate(fa_question, dest='en').text
    bt_questions.append(bt_question)
    bt_answer = translator.translate(fa_answer, dest='en').text
    bt_answers.append(bt_answer)
    if (index - bgn) % bsize == 0 and not index - bgn == 0 :
       question_Ps, question_Rs, question_F1s = bert_score.score(bt_questions, questions, lang='en', device=device, verbose=True)
       answer_Ps, answer_Rs, answer_F1s = bert_score.score(bt_answers, answers, lang='en', device=device, verbose=True)
       for i in range(bsize):
           append_record_to_excel('translated_hcm.xlsx', questions[i],answers[i],
                                  fa_questions[i],fa_answers[i],bt_questions[i],
                                  bt_answers[i],question_Ps.numpy()[i],
                                  question_Rs.numpy()[i],question_F1s.numpy()[i],
                                  answer_Ps.numpy()[i], answer_Rs.numpy()[i],
                                  answer_F1s.numpy()[i])
       questions = []
       answers = []
       fa_questions = []
       fa_answers = []
       bt_questions = []
       bt_answers = []